In [2]:
import  tensorflow as tf
from    tensorflow.keras import datasets, layers, optimizers, Sequential, metrics


def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(y, dtype=tf.int32)

    return x,y 

In [3]:
batchsz = 128
(x, y), (x_val, y_val) = datasets.mnist.load_data()
print('datasets:', x.shape, y.shape, x.min(), x.max())

datasets: (60000, 28, 28) (60000,) 0 255


In [9]:
set(y.tolist())

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [6]:
db = tf.data.Dataset.from_tensor_slices((x,y))
db = db.map(preprocess).shuffle(60000).batch(batchsz).repeat(10)

ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
ds_val = ds_val.map(preprocess).batch(batchsz) 

In [10]:
network = Sequential([layers.Dense(256, activation='relu'),
                     layers.Dense(128, activation='relu'),
                     layers.Dense(64, activation='relu'),
                     layers.Dense(32, activation='relu'),
                     layers.Dense(10)])
network.build(input_shape=(None, 28*28))
network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  200960    
_________________________________________________________________
dense_1 (Dense)              multiple                  32896     
_________________________________________________________________
dense_2 (Dense)              multiple                  8256      
_________________________________________________________________
dense_3 (Dense)              multiple                  2080      
_________________________________________________________________
dense_4 (Dense)              multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


In [12]:
optimizer = optimizers.Adam(lr=0.01)

acc_meter = metrics.Accuracy()
loss_meter = metrics.Mean()

In [13]:
for step, (x,y) in enumerate(db):

    with tf.GradientTape() as tape:
        # [b, 28, 28] => [b, 784]
        x = tf.reshape(x, (-1, 28*28))
        # [b, 784] => [b, 10]
        out = network(x)
        # [b] => [b, 10]
        y_onehot = tf.one_hot(y, depth=10) 
        # [b]
        loss = tf.reduce_mean(tf.losses.categorical_crossentropy(y_onehot, out, from_logits=True))

        loss_meter.update_state(loss)

 

    grads = tape.gradient(loss, network.trainable_variables)
    optimizer.apply_gradients(zip(grads, network.trainable_variables))


    if step % 100 == 0:

        print(step, 'loss:', loss_meter.result().numpy()) 
        loss_meter.reset_states()


    # evaluate
    if step % 500 == 0:
        total, total_correct = 0., 0
        acc_meter.reset_states()

        for step, (x, y) in enumerate(ds_val): 
            # [b, 28, 28] => [b, 784]
            x = tf.reshape(x, (-1, 28*28))
            # [b, 784] => [b, 10]
            out = network(x) 


            # [b, 10] => [b] 
            pred = tf.argmax(out, axis=1) 
            pred = tf.cast(pred, dtype=tf.int32)
            # bool type 
            correct = tf.equal(pred, y)
            # bool tensor => int tensor => numpy
            total_correct += tf.reduce_sum(tf.cast(correct, dtype=tf.int32)).numpy()
            total += x.shape[0]

            acc_meter.update_state(y, pred)


        print(step, 'Evaluate Acc:', total_correct/total, acc_meter.result().numpy())


0 loss: 2.3430045
78 Evaluate Acc: 0.2143 0.2143
100 loss: 0.5216492
200 loss: 0.2301424
300 loss: 0.20898262
400 loss: 0.199205
500 loss: 0.16166781
78 Evaluate Acc: 0.9582 0.9582
600 loss: 0.14944176
700 loss: 0.13741818
800 loss: 0.13429566
900 loss: 0.124954216
1000 loss: 0.11625112
78 Evaluate Acc: 0.9675 0.9675
1100 loss: 0.10145916
1200 loss: 0.10475279
1300 loss: 0.106985204
1400 loss: 0.10870165
1500 loss: 0.08039901
78 Evaluate Acc: 0.9702 0.9702
1600 loss: 0.1086403
1700 loss: 0.089237764
1800 loss: 0.10264205
1900 loss: 0.093836695
2000 loss: 0.069580704
78 Evaluate Acc: 0.9703 0.9703
2100 loss: 0.08971125
2200 loss: 0.08915021
2300 loss: 0.079365686
2400 loss: 0.0751986
2500 loss: 0.074397795
78 Evaluate Acc: 0.9611 0.9611
2600 loss: 0.09853884
2700 loss: 0.08801999
2800 loss: 0.10047382
2900 loss: 0.080029964
3000 loss: 0.07832683
78 Evaluate Acc: 0.9693 0.9693
3100 loss: 0.07068557
3200 loss: 0.08225421
3300 loss: 0.074772075
3400 loss: 0.063054346
3500 loss: 0.070554934